<a href="https://colab.research.google.com/github/Nik8x/Pyspark/blob/master/SparkSQL_911_Emergency_Helpline_Number_Data_Analysis/SparkSQL_911_Emergency_Helpline_Number_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

import pandas as pd
import matplotlib.pyplot as plt

# from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
# from pyspark.ml.regression import RandomForestRegressor, LinearRegression
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
# from pyspark.ml import Pipeline, Model
from pyspark.sql.functions import col

In [0]:
from pyspark.sql import SQLContext
sql_context = SQLContext(sc)

In [0]:
df = spark.read.format("csv").options(header="true").load("911.csv")

In [0]:
type(df)

pyspark.sql.dataframe.DataFrame

In [0]:
df.show(5)

+----------+-----------+--------------------+-----+--------------------+-------------------+-----------------+--------------------+---+
|       lat|        lng|                desc|  zip|               title|          timeStamp|              twp|                addr|  e|
+----------+-----------+--------------------+-----+--------------------+-------------------+-----------------+--------------------+---+
|40.2978759|-75.5812935|REINDEER CT & DEA...|19525|EMS: BACK PAINS/I...|2015-12-10 17:40:00|      NEW HANOVER|REINDEER CT & DEA...|  1|
|40.2580614|-75.2646799|BRIAR PATH & WHIT...|19446|EMS: DIABETIC EME...|2015-12-10 17:40:00|HATFIELD TOWNSHIP|BRIAR PATH & WHIT...|  1|
|40.1211818|-75.3519752|HAWS AVE; NORRIST...|19401| Fire: GAS-ODOR/LEAK|2015-12-10 17:40:00|       NORRISTOWN|            HAWS AVE|  1|
|40.1161530|-75.3435130|AIRY ST & SWEDE S...|19401|EMS: CARDIAC EMER...|2015-12-10 17:40:01|       NORRISTOWN|  AIRY ST & SWEDE ST|  1|
|40.2514920|-75.6033497|CHERRYWOOD CT & D...| nu

In [0]:
data = sc.textFile("911.csv")

In [0]:
print(type(data))

print(type(data.take(5)))

data.take(5)

<class 'pyspark.rdd.RDD'>
<class 'list'>


['lat,lng,desc,zip,title,timeStamp,twp,addr,e',
 '40.2978759,-75.5812935,REINDEER CT & DEAD END;  NEW HANOVER; Station 332; 2015-12-10 @ 17:10:52;,19525,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1',
 '40.2580614,-75.2646799,BRIAR PATH & WHITEMARSH LN;  HATFIELD TOWNSHIP; Station 345; 2015-12-10 @ 17:29:21;,19446,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1',
 '40.1211818,-75.3519752,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-Station:STA27;,19401,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1',
 '40.1161530,-75.3435130,AIRY ST & SWEDE ST;  NORRISTOWN; Station 308A; 2015-12-10 @ 16:47:36;,19401,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST,1']

In [0]:
header = data.first()
print(type(header))
header

<class 'str'>


'lat,lng,desc,zip,title,timeStamp,twp,addr,e'

In [0]:
data1 = data.filter(lambda row: row != header)
data1.take(2)

['40.2978759,-75.5812935,REINDEER CT & DEAD END;  NEW HANOVER; Station 332; 2015-12-10 @ 17:10:52;,19525,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1',
 '40.2580614,-75.2646799,BRIAR PATH & WHITEMARSH LN;  HATFIELD TOWNSHIP; Station 345; 2015-12-10 @ 17:29:21;,19446,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1']

In [0]:
data1.map(lambda x: x.split(",")).take(1)#.filter(x => x.length>=9).map(x => emergency(x(0),x(1),x(2),x(3),x(4).substring(0 , x(4).indexOf(":")),x(5),x(6),x(7),x(8))).toDF

[['40.2978759',
  '-75.5812935',
  'REINDEER CT & DEAD END;  NEW HANOVER; Station 332; 2015-12-10 @ 17:10:52;',
  '19525',
  'EMS: BACK PAINS/INJURY',
  '2015-12-10 17:40:00',
  'NEW HANOVER',
  'REINDEER CT & DEAD END',
  '1']]

In [0]:
data1.map(lambda x: x.split(",")).filter(lambda x: len(x) >= 9).take(1)

[['40.2978759',
  '-75.5812935',
  'REINDEER CT & DEAD END;  NEW HANOVER; Station 332; 2015-12-10 @ 17:10:52;',
  '19525',
  'EMS: BACK PAINS/INJURY',
  '2015-12-10 17:40:00',
  'NEW HANOVER',
  'REINDEER CT & DEAD END',
  '1']]

In [0]:
data1.map(lambda x: x.split(",")).filter(lambda x: len(x) >= 9).take(1)[0][4][0: data1.map(lambda x: x.split(",")).filter(lambda x: len(x) >= 9).take(1)[0][4].index(":")]

'EMS'

In [0]:
data1.map(lambda x: x.split(",")).filter(lambda x: len(x) >= 9).map(lambda x: Row(lat = x[0], lng = x[1], desc = x[2], zip_code = x[3], title = x[4][0 : x[4].index(":")], timeStamp = x[5], twp = x[6], addr = x[7], e = x[8])).take(2)

[Row(addr='REINDEER CT & DEAD END', desc='REINDEER CT & DEAD END;  NEW HANOVER; Station 332; 2015-12-10 @ 17:10:52;', e='1', lat='40.2978759', lng='-75.5812935', timeStamp='2015-12-10 17:40:00', title='EMS', twp='NEW HANOVER', zip_code='19525'),
 Row(addr='BRIAR PATH & WHITEMARSH LN', desc='BRIAR PATH & WHITEMARSH LN;  HATFIELD TOWNSHIP; Station 345; 2015-12-10 @ 17:29:21;', e='1', lat='40.2580614', lng='-75.2646799', timeStamp='2015-12-10 17:40:00', title='EMS', twp='HATFIELD TOWNSHIP', zip_code='19446')]

In [0]:
emergency_data = data1.map(lambda x: x.split(",")).filter(lambda x: len(x) >= 9).map(lambda x: Row(lat = x[0], lng = x[1], desc = x[2], zip_code = x[3], title = x[4][0 : x[4].index(":")], timeStamp = x[5], twp = x[6], addr = x[7], e = x[8])).toDF()
emergency_data

DataFrame[addr: string, desc: string, e: string, lat: string, lng: string, timeStamp: string, title: string, twp: string, zip_code: string]

In [0]:
emergency_data.show(2)

+--------------------+--------------------+---+----------+-----------+-------------------+-----+-----------------+--------+
|                addr|                desc|  e|       lat|        lng|          timeStamp|title|              twp|zip_code|
+--------------------+--------------------+---+----------+-----------+-------------------+-----+-----------------+--------+
|REINDEER CT & DEA...|REINDEER CT & DEA...|  1|40.2978759|-75.5812935|2015-12-10 17:40:00|  EMS|      NEW HANOVER|   19525|
|BRIAR PATH & WHIT...|BRIAR PATH & WHIT...|  1|40.2580614|-75.2646799|2015-12-10 17:40:00|  EMS|HATFIELD TOWNSHIP|   19446|
+--------------------+--------------------+---+----------+-----------+-------------------+-----+-----------------+--------+
only showing top 2 rows



In [0]:
print(type(emergency_data))
emergency_data.count()

<class 'pyspark.sql.dataframe.DataFrame'>


99492

In [0]:
emergency_data.printSchema()

root
 |-- addr: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- e: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- timeStamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- twp: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [0]:
emergency_data.registerTempTable("emergency_911")

In [0]:
data2 = sc.textFile("zipcode.csv")
header1 = data2.first()
data3 = data2.filter(lambda row: row != header1)

In [0]:
data2.take(2)

['"zip","city","state","latitude","longitude","timezone","dst"',
 '"00210","Portsmouth","NH","43.005895","-71.013202","-5","1"']

In [0]:
data3.take(2)

['"00210","Portsmouth","NH","43.005895","-71.013202","-5","1"',
 '"00211","Portsmouth","NH","43.005895","-71.013202","-5","1"']

In [0]:
data3.map(lambda x: x.split(",")).map(lambda x: Row(zip_code = x[0].replace('"', ""), city = x[1].replace('"', ""), state = x[2].replace('"', ""), latitude = x[3].replace('"', ""), longitude = x[4].replace('"', ""), timezone = x[5].replace('"', ""), dst = x[6].replace('"', ""))).take(1)

[Row(city='Portsmouth', dst='1', latitude='43.005895', longitude='-71.013202', state='NH', timezone='-5', zip_code='00210')]

In [0]:
zipcodes = data3.map(lambda x: x.split(",")).map(lambda x: Row(zip_code = x[0].replace('"', ""), city = x[1].replace('"', ""), state = x[2].replace('"', ""), latitude = x[3].replace('"', ""), longitude = x[4].replace('"', ""), timezone = x[5].replace('"', ""), dst = x[6].replace('"', ""))).toDF()
zipcodes.show(2)

+----------+---+---------+----------+-----+--------+--------+
|      city|dst| latitude| longitude|state|timezone|zip_code|
+----------+---+---------+----------+-----+--------+--------+
|Portsmouth|  1|43.005895|-71.013202|   NH|      -5|   00210|
|Portsmouth|  1|43.005895|-71.013202|   NH|      -5|   00211|
+----------+---+---------+----------+-----+--------+--------+
only showing top 2 rows



In [0]:
zipcodes.count(), type(zipcodes)

(43191, pyspark.sql.dataframe.DataFrame)

In [0]:
zipcodes.createOrReplaceTempView("zipcode_table")

In [0]:
type(sql_context), type(sql_context.sql)

(pyspark.sql.context.SQLContext, method)

In [0]:
print(type(sql_context.sql("select * from emergency_911")))
sql_context.sql("select * from emergency_911").show(2)

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+--------------------+---+----------+-----------+-------------------+-----+-----------------+--------+
|                addr|                desc|  e|       lat|        lng|          timeStamp|title|              twp|zip_code|
+--------------------+--------------------+---+----------+-----------+-------------------+-----+-----------------+--------+
|REINDEER CT & DEA...|REINDEER CT & DEA...|  1|40.2978759|-75.5812935|2015-12-10 17:40:00|  EMS|      NEW HANOVER|   19525|
|BRIAR PATH & WHIT...|BRIAR PATH & WHIT...|  1|40.2580614|-75.2646799|2015-12-10 17:40:00|  EMS|HATFIELD TOWNSHIP|   19446|
+--------------------+--------------------+---+----------+-----------+-------------------+-----+-----------------+--------+
only showing top 2 rows



In [0]:
print(type(sql_context.sql("select * from zipcode_table")))
sql_context.sql("select * from zipcode_table").show(2)

<class 'pyspark.sql.dataframe.DataFrame'>
+----------+---+---------+----------+-----+--------+--------+
|      city|dst| latitude| longitude|state|timezone|zip_code|
+----------+---+---------+----------+-----+--------+--------+
|Portsmouth|  1|43.005895|-71.013202|   NH|      -5|   00210|
|Portsmouth|  1|43.005895|-71.013202|   NH|      -5|   00211|
+----------+---+---------+----------+-----+--------+--------+
only showing top 2 rows



In [0]:
build1 = sql_context.sql("select e.title, z.city,z.state from emergency_911 e join zipcode_table z on e.zip_code = z.zip_code")
build1.show(5)

+-------+-------+-----+
|  title|   city|state|
+-------+-------+-----+
|    EMS|Wyncote|   PA|
|    EMS|Wyncote|   PA|
|Traffic|Wyncote|   PA|
|   Fire|Wyncote|   PA|
|Traffic|Wyncote|   PA|
+-------+-------+-----+
only showing top 5 rows



In [0]:
type(build1), build1.count()

(pyspark.sql.dataframe.DataFrame, 86637)

In [0]:
# What kind of problems are prevalent, and in which state?

In [0]:
ps = build1.rdd.map(lambda x: (x[0] + " -->" + str(x[2]))) # .rdd because dataframe does not have map function
ps.take(2)

['EMS -->PA', 'EMS -->PA']

In [0]:
ps.map(lambda x: (x,1)).take(2)#.reduceByKey(_+_).map(item => item.swap).sortByKey(false).foreach(println)

[('EMS -->PA', 1), ('EMS -->PA', 1)]

In [0]:
type(ps.map(lambda x: (x,1)).take(2)[0])

tuple

In [0]:
from operator import add

In [0]:
ps.map(lambda x: (x,1)).reduceByKey(add).take(2)#.map(item => item.swap).sortByKey(false).foreach(println)

[('Traffic -->PA', 29297), ('Traffic -->AL', 1)]

In [0]:
ps.map(lambda x: (x,1)).reduceByKey(add).map(lambda item: (item[1], item[0])).take(2)#.sortByKey(false).foreach(println)

[(29297, 'Traffic -->PA'), (1, 'Traffic -->AL')]

In [0]:
ps.map(lambda x: (x,1)).reduceByKey(add).map(lambda item: (item[1], item[0])).sortByKey(False).take(2)#.foreach(println)

[(44326, 'EMS -->PA'), (29297, 'Traffic -->PA')]

In [0]:
ps1 = ps.map(lambda x: (x,1)).reduceByKey(add).map(lambda item: (item[1], item[0])).sortByKey(False)
type(ps1)

pyspark.rdd.PipelinedRDD

In [0]:
ps1.collect()[0:3]

[(44326, 'EMS -->PA'), (29297, 'Traffic -->PA'), (13012, 'Fire -->PA')]

In [0]:
# What kind of problems are prevalent, and in which city?

In [0]:
ps2 = build1.rdd.map(lambda x: (x[0] + " -->" + str(x[1])))
ps2.take(2)

['EMS -->Wyncote', 'EMS -->Wyncote']

In [0]:
ps3 = ps2.map(lambda x: (x,1)).reduceByKey(add).map(lambda item: (item[1], item[0])).sortByKey(False)
ps3.collect()[0:10]

[(7409, 'EMS -->Norristown'),
 (4237, 'EMS -->Pottstown'),
 (2919, 'Traffic -->Norristown'),
 (2665, 'EMS -->Lansdale'),
 (1822, 'Traffic -->Pottstown'),
 (1677, 'EMS -->King Of Prussia'),
 (1505, 'Fire -->Norristown'),
 (1466, 'Traffic -->Lansdale'),
 (1451, 'EMS -->Royersford'),
 (1397, 'EMS -->Ambler')]